In [20]:
import pandas as pd
import numpy as np
import math
import time
import os

In [21]:
LARGE_DATASET = False

In [22]:
if LARGE_DATASET:
    movie_path = "ml-latest/movies.csv"
    tags_path = "ml-latest/tags.csv"
    ratings_path = "ml-latest/ratings.csv"
    movie_profiles_path = "/media/karol/E8B625B3B6258364/calculated/movie_profiles"
    user_profiles_path = "/media/karol/E8B625B3B6258364/calculated/user_profiles/"
    movie_matrix_path = "/media/karol/E8B625B3B6258364/calculated/movie_matrix"
else:
    movie_path = "ml-latest-small/movies.csv"
    tags_path = "ml-latest-small/tags.csv"
    ratings_path = "ml-latest-small/split/ratings_testing.csv"
    movie_profiles_path = "/media/karol/E8B625B3B6258364/calculated-small/movie_profiles"
    user_profiles_path = "/media/karol/E8B625B3B6258364/calculated-small/user_profiles/"

In [23]:
def read_user_profile(userId):
    directory = user_profiles_path + str(userId // 1000) + "/" + str(userId)
    user_profile = pd.read_csv(directory,  header=None, usecols=[1,2,3])
    return user_profile.rename(columns={1:"tag",2:"u_weight",3:"userId"})

def get_movie_profile(movieId):
    return movie_profiles[movie_profiles['movieId'] == movieId]

# Predict user's rating on test data

Finally a method fast enough to process large dataset

In [24]:
def get_movie_profile_matrix(movie_indices):
    if len(movie_indices) == 1:
        mid = movie_indices.iloc[0]
        return get_movie_profile(mid).drop(['movieId'],1).rename(columns={"m_weight": mid})
    
    matrix_left = get_movie_profile_matrix(movie_indices[:len(movie_indices)//2])
    matrix_right = get_movie_profile_matrix(movie_indices[len(movie_indices)//2:])
    matrix = pd.merge(matrix_left,matrix_right , on='tag', how='outer', sort=False)
    return matrix

def get_similarity(userId, movies_df):
    i = read_user_profile(userId)
    i['u_weight'] = i['u_weight'] / np.sqrt(np.square(i['u_weight']).sum())
    similarity = pd.merge(i, movies_df, on='tag', how='outer', sort=False)
    return similarity.iloc[:,3:].mul(i['u_weight'],0).sum(0)

In [25]:
mf = open(movie_profiles_path, 'r')
movie_profiles = pd.read_csv(mf).rename(columns={'weight':'m_weight'})

movies = movie_profiles['movieId'].drop_duplicates()

In [26]:
movie_matrix = get_movie_profile_matrix(movies)

In [27]:
ratings_data = pd.read_csv(ratings_path)

In [29]:
ratings_data['userId'].drop_duplicates()

0         4
4         8
8        15
12       17
16       19
20       21
24       22
28       23
32       26
36       30
40       33
44       34
48       36
52       38
56       41
60       43
64       48
68       56
72       57
76       61
80       67
84       68
88       72
92       73
96       75
100      77
104      78
108      81
112      83
116      84
       ... 
912     592
916     595
920     596
924     597
928     598
932     599
936     602
940     603
944     605
948     607
952     608
956     609
960     615
964     620
968     623
972     624
976     626
980     627
984     641
988     646
992     647
996     648
1000    652
1004    654
1008    655
1012    656
1016    659
1020    664
1024    665
1028    671
Name: userId, Length: 258, dtype: int64

In [31]:
start = time.time()
for userId in ratings_data['userId'].drop_duplicates():
        f = open("calculated/content_based_recommendations/" + str(userId), "w")
        recomendations = pd.DataFrame()
        recomendations['movie'] = get_similarity(userId,movie_matrix).sort_values(ascending=False).index
        recomendations.to_csv(f, header=None ,index=False)
print(time.time() - start)

2.8426177501678467


# Small Dataset processed in less than 4 seconds



in big dataset every user is calculated in less than second

TODO:

    ogarnąć dokumentacje, spostrzeżenia itp